In [1]:
import numpy as np
# from sklearn.cross_validation import train_test_split
from sklearn.model_selection import train_test_split

def get_data():
    '''
    获取数据
    :return: 文本数据，对应的labels
    '''
    with open('../../../chapter-9/classification/data/ham_data.txt', 'r', encoding='UTF-8') as ham_f, \
    open('../../../chapter-9/classification/data/spam_data.txt', 'r', encoding='UTF-8') as spam_f:
        ham_data = ham_f.readlines()
        spam_data = spam_f.readlines()
        
        ham_label = np.ones(len(ham_data)).tolist()
        spam_label = np.zeros(len(spam_data)).tolist()
        
        corpus = ham_data + spam_data
        labels = ham_label + spam_label
        
    return corpus, labels

def prepare_datasets(corpus, labels, test_data_proportion=0.3):
    '''
    :param corpus: 文本数据
    :param labels: label数据
    :param test_data_proportion:测试数据占比 
    :return: 训练数据,测试数据，训练label,测试label
    '''
    train_X, test_X, train_Y, test_Y = \
    train_test_split(corpus, labels, test_size=test_data_proportion, random_state=42)
    return train_X, test_X, train_Y, test_Y

def remove_empty_docs(corpus, labels):
    filtered_corpus = []
    filtered_labels = []
    for doc, label in zip(corpus, labels):
        if doc.strip():
            filtered_corpus.append(doc)
            filtered_labels.append(label)
    return filtered_corpus, filtered_labels

In [2]:
from sklearn import metrics

def get_metrics(true_labels, predicted_labels):
    print('accuracy: ', np.round(metrics.accuracy_score(
        true_labels, predicted_labels), 2))
    print('precision: ', np.round(metrics.precision_score(
        true_labels,predicted_labels, average='weighted'),2))
    print('recall: ', np.round(metrics.recall_score(
        true_labels, predicted_labels, average='weighted'), 2))
    print('F1: ', np.round(metrics.f1_score(
        true_labels, predicted_labels, average='weighted'), 2))

def train_predict_evaluate_model(classifier, train_features, train_labels,
                                test_features, test_labels):
    # build model
    classifier.fit(train_features, train_labels)
    # predict using model
    predictions = classifier.predict(test_features)
    # evaluate model prediction performance
    get_metrics(true_labels=test_labels, predicted_labels=predictions)
    return predictions

In [5]:
corpus, labels = get_data()
print('总的数据量: ', len(labels))
    
corpus, labels = remove_empty_docs(corpus, labels)
    
print('样本之一: ', corpus[10])
print('样本的label: ', labels[10])
label_name_map = ['垃圾邮件', '正常邮件']
print('实际类型: ', label_name_map[int(labels[10])])
    
train_corpus, test_corpus, train_labels, test_labels = \
prepare_datasets(corpus, labels, test_data_proportion=0.3)

总的数据量:  10001
样本之一:  北京售票员可厉害，嘿嘿，有专座的，会直接拉着脖子指着鼻子让上面的人站起来让 座的，呵呵，比较赞。。。 杭州就是很少有人给让座，除非司机要求乘客那样做。 五一去杭州一个景点玩，车上有两个不到一岁的小孩，就是没有人给让座，没办法家长只能在车上把小孩的推车打开让孩子坐进去，但是孩子还是闹，只能抱着，景点离市区很远，车上很颠，最后家长坐在地上抱孩子，就是没有一个人给让座，要是在北京，一上车就有人让座了

样本的label:  1.0
实际类型:  正常邮件


In [6]:
from normalization import normalize_corpus
    
# 进行归一化
norm_train_corpus = normalize_corpus(train_corpus)
norm_test_corpus = normalize_corpus(test_corpus)
    
#     ''.strip()

In [7]:
from feature_extractors import bow_extractor, tfidf_extractor
import gensim
import jieba
    
# 词袋模型特征
bow_vectorizer, bow_train_features = bow_extractor(norm_train_corpus)
bow_test_features = bow_vectorizer.transform(norm_test_corpus)

# tfidf 特征
tfidf_vectorizer, tfidf_train_features = tfidf_extractor(norm_train_corpus)
tfidf_test_features = tfidf_vectorizer.transform(norm_test_corpus)

# tokenize documents
tokenized_train = [jieba.lcut(text) for text in norm_train_corpus]
print(tokenized_train[2:10])
tokenized_test = [jieba.lcut(text) for text in norm_test_corpus]
    
# build word2vec 模型
model = gensim.models.Word2Vec(tokenized_train, size=500, window=100,
                                min_count=30, sample=1e-3)

[['10156', '说', '的', '呵呵', '标题', 'Re', '我', '要是', '你', '女朋友', '绝对', '跟', '你', '分手', 'Re', '昨晚', '猫', '又', '闹', '了', '一夜', '嗯', '，', '谁', '说', '我', '养猫', '是因为', '有', '爱心', '我', '跟', '谁', '急', '是', '哈', '，', '不是', '用', '爱心', '区分', '的', '喜欢', '宠物', '的', '就', '养', '，', '不', '喜欢', '的', '就', '不养', '呗', '卷', '卷', '，', '你', '搞', '成', '这', '副', '鬼', '样子', '，', '还', '好意思', '来', '找', '我', '撒娇', '。', '。', '。'], ['中信', '（', '国际', '）', '电子科技', '有限公司', '推出', '新', '产品', '：', '升职', '步步高', '、', '做生意', '发大财', '、', '连', '找', '情人', '都', '用', '的', '上', '，', '详情', '进入', '网址', 'httpwwwusa5588comccc', '电话', '：', '02033770208', '服务', '热线', '：', '013650852999'], ['贵', '公司', '负责人', '：', '你好', '！', '本', '公司', '祥泰', '实业', '有限公司', '）', '具有', '进出口', '及', '国内贸易', '的', '企业', '承', '多家', '公司', '委托', '有', '广告', '建筑工程', '其它', '服务', '商品销售', '等', '的', '发票', '向', '外代', '开', '点数', '优惠', '本', '公司', '原则', '是', '满意', '后', '付款', '有意者', '请来', '电', '洽谈', '电话', '：', '013631690076', '邮箱', '：', 'shitailong8163com', '联系人', '：', '郭生', '如

In [8]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression

mnb = MultinomialNB()
svm = SGDClassifier(loss='hinge', max_iter=100) # 书上用的是n_iter会报错
lr = LogisticRegression()

# 基于词袋模型的多项朴素贝叶斯
print("基于词袋模型特征的贝叶斯分类器")
mnb_bow_predictions = train_predict_evaluate_model(classifier=mnb,
                                                   train_features=bow_train_features,
                                                   train_labels=train_labels,
                                                   test_features=bow_test_features,
                                                   test_labels=test_labels)

# 基于词袋模型特征的逻辑回归
print("基于词袋模型特征的逻辑回归")
lr_bow_predictions = train_predict_evaluate_model(classifier=lr,
                                                  train_features=bow_train_features,
                                                  train_labels=train_labels,
                                                  test_features=bow_test_features,
                                                  test_labels=test_labels)

# 基于词袋模型的支持向量机方法
print("基于词袋模型的支持向量机")
svm_bow_predictions = train_predict_evaluate_model(classifier=svm,
                                                   train_features=bow_train_features,
                                                   train_labels=train_labels,
                                                   test_features=bow_test_features,
                                                   test_labels=test_labels)


# 基于tfidf的多项式朴素贝叶斯模型
print("基于tfidf的贝叶斯模型")
mnb_tfidf_predictions = train_predict_evaluate_model(classifier=mnb,
                                                     train_features=tfidf_train_features,
                                                     train_labels=train_labels,
                                                     test_features=tfidf_test_features,
                                                     test_labels=test_labels)
# 基于tfidf的逻辑回归模型
print("基于tfidf的逻辑回归模型")
lr_tfidf_predictions=train_predict_evaluate_model(classifier=lr,
                                                     train_features=tfidf_train_features,
                                                     train_labels=train_labels,
                                                     test_features=tfidf_test_features,
                                                     test_labels=test_labels)


# 基于tfidf的支持向量机模型
print("基于tfidf的支持向量机模型")
svm_tfidf_predictions = train_predict_evaluate_model(classifier=svm,
                                                     train_features=tfidf_train_features,
                                                     train_labels=train_labels,
                                                     test_features=tfidf_test_features,
                                                     test_labels=test_labels)

基于词袋模型特征的贝叶斯分类器
accuracy:  0.79
precision:  0.85
recall:  0.79
F1:  0.78
基于词袋模型特征的逻辑回归
accuracy:  0.96
precision:  0.96
recall:  0.96
F1:  0.96
基于词袋模型的支持向量机
accuracy:  0.97
precision:  0.97
recall:  0.97
F1:  0.97
基于tfidf的贝叶斯模型
accuracy:  0.79
precision:  0.85
recall:  0.79
F1:  0.78
基于tfidf的逻辑回归模型
accuracy:  0.94
precision:  0.94
recall:  0.94
F1:  0.94
基于tfidf的支持向量机模型
accuracy:  0.97
precision:  0.97
recall:  0.97
F1:  0.97


D:\alex_space\software\anaconda\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
D:\alex_space\software\anaconda\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [9]:
import re

num = 0
for document, label, predicted_label in zip(test_corpus, test_labels, svm_tfidf_predictions):
    if label == 0 and predicted_label == 0:
        print('邮件类型:', label_name_map[int(label)])
        print('预测的邮件类型:', label_name_map[int(predicted_label)])
        print('文本:-')
        print(re.sub('\n', ' ', document))

        num += 1
        if num == 4:
            break

num = 0
for document, label, predicted_label in zip(test_corpus, test_labels, svm_tfidf_predictions):
    if label == 1 and predicted_label == 0:
        print('邮件类型:', label_name_map[int(label)])
        print('预测的邮件类型:', label_name_map[int(predicted_label)])
        print('文本:-')
        print(re.sub('\n', ' ', document))

        num += 1
        if num == 4:
            break

邮件类型: 垃圾邮件
预测的邮件类型: 垃圾邮件
文本:-
中信（国际）电子科技有限公司推出新产品： 升职步步高、做生意发大财、连找情人都用的上，详情进入 网  址:  http://www.usa5588.com/ccc 电话：020-33770208   服务热线：013650852999 
邮件类型: 垃圾邮件
预测的邮件类型: 垃圾邮件
文本:-
您好！ 我公司有多余的发票可以向外代开！（国税、地税、运输、广告、海关缴款书）。 如果贵公司（厂）有需要请来电洽谈、咨询！ 联系电话: 013510251389  陈先生 谢谢 顺祝商祺! 
邮件类型: 垃圾邮件
预测的邮件类型: 垃圾邮件
文本:-
如果您在信箱中不能正常阅读此邮件，请点击这里 
邮件类型: 垃圾邮件
预测的邮件类型: 垃圾邮件
文本:-
以下不能正确显示请点此 IFRAME: http://bbs.ewzw.com/viewthread.php?tid=3790 
邮件类型: 正常邮件
预测的邮件类型: 垃圾邮件
文本:-
你好，我30，未婚，研究生毕业，想找一个在校的女孩做朋友，每月可以给她2000元零用 我不会走入她的生活中去，为她保密 qq ******* 回了一封信以后： 阿，不愿意跟在一群人后面买单，那会让人觉得特傻 只想一对一交往 【 在 leeann2002 的来信中提到: 】 啊,那您介意找一堆女孩子做朋友吗? 也不用给零用钱拉~~用来带我们一起玩就可以了~,唱歌跳舞什么的~~ (要不2000就把自己卖了...) 有照片连接吗? 
邮件类型: 正常邮件
预测的邮件类型: 垃圾邮件
文本:-
公司：集制作，创作以及宣传为一体的音乐制作文化发展公司 拥有录音棚，主要制作唱片， 也涉及电影、电视剧、广告等音乐制作及创作 职位：经理助理（说文秘也行） 要求：女   22～25 聪明本分 有一定的协调和办事能力 长相过得去 工作业务范围： 平时管理公司文件 接待、电话、上网 负责安排歌手及制作人的工作下达 待遇：试用期月薪1000 转正1500～2000，另有提成 公司地址：北京市朝阳区麦子店 电话留了会被封么？ 先信箱联系吧 
邮件类型: 正常邮件
预测的邮件类型: 垃圾邮件
文本:-
现已确定2006年度不会进行研究生培养机制改革试点，我校研究生招生类别、培养费用 等相关政策仍按现行规

## 草稿

In [1]:
with open('../../../chapter-9/classification/data/ham_data.txt', encoding='UTF-8') as ham_f:
    print(ham_f.readline())

讲的是孔子后人的故事。一个老领导回到家乡，跟儿子感情不和，跟贪财的孙子孔为本和睦。 老领导的弟弟魏宗万是赶马车的。 有个洋妞大概是考察民俗的，在他们家过年。 孔为本总想出国，被爷爷教育了。 最后，一家人基本和解。 顺便问另一类电影，北京青年电影制片厂的。中越战背景。一军人被介绍了一个对象，去相亲。女方是军队医院的护士，犹豫不决，总是在回忆战场上负伤的男友，好像还没死。最后 男方表示理解，归队了。



In [15]:
import jieba
import re
import string
def remove_special_characters(text):
    tokens = tokenize_text(text)
    pattern = re.compile('[{}]'.format(re.escape(string.punctuation)))
    filtered_tokens = filter(None, [pattern.sub('', token) for token in tokens])
def tokenize_text(text):
    tokens = jieba.cut(text)
    print(tokens)
    tokens = [token.strip() for token in tokens]
    print(tokens)
    return tokens
remove_special_characters('我最近很好~！@你呢？')

<generator object Tokenizer.cut at 0x000002141D3DC0C0>
['我', '最近', '很', '好', '~', '！', '@', '你', '呢', '？']


TypeError: filter expected 2 arguments, got 1